In [ ]:
import dotenv
import json
import os
import requests
import sys
from rich.progress import track
from concurrent.futures import ThreadPoolExecutor, as_completed

import censusgeocode

from census import Census
from us import states

dotenv.load_dotenv()


CURRENT_DIR = %pwd
ROOT_DIR = os.path.dirname(CURRENT_DIR)

SRC_DIR = os.path.join(ROOT_DIR, 'src')
sys.path.append(SRC_DIR)

CENSUS_API_KEY = os.environ["CENSUS_API_KEY"]
GOOGLE_API_KEY = os.environ["GOOGLE_MAPS_PLACES_API_KEY"]


In [ ]:
from search_context.us_census import USCensusAPI
from search_context.google_places import GooglePlacesAPI


In [ ]:
api = USCensusAPI(CENSUS_API_KEY, verbose=True)
api.warm_cache()
api.get_census_data_from_address("B01003_001E", "2685 California Street, Mountain View, CA")

In [ ]:

api.get_census_data_from_address(["B01003_001E", "B01001_002E"], "2685 California Street, Mountain View, CA")


In [ ]:
c = Census(CENSUS_API_KEY)
c.acs5.get(('NAME', 'B25034_010E'),
          {'for': 'state:{}'.format(states.MD.fips)})

In [ ]:
tables = c.acs5.tables()

In [ ]:
censusgeocode.addressbatch("./data/batchaddress.csv")

In [ ]:
c.acs5.groups_url % ("2020", "acs5")

In [ ]:
len(tables)

In [ ]:
fmt_url_old = c.acs5.groups_url % ("2020", "acs5")
fmt_url = fmt_url_old.rsplit('.', 1)[0]
fmt_url = os.path.join(fmt_url, "{}.json")

def fetch_url(url):
    print(url)
    try:
        response = requests.get(url, timeout=5.0)
    except Exception as e:
        print(f"Fail: {e}")
        return None
    return response.json()


table_options = {}
urls = [fmt_url.format(group["name"]) for group in tables]

futures = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(fetch_url, url) for url in urls]


In [ ]:
for future in as_completed(futures):
    try:
        result = future.result()
    except:
        continue
    if result is None:
        print(f"Failed {url}")
        continue
    print(result)
    table_options.update(result["variables"])

In [ ]:
with open("census_codes.json", "w") as outfile:
    json.dump(table_options, outfile, indent=4)


In [ ]:
len(table_options.keys())

In [ ]:
c.sf1.get('NAME', geo={'for': 'tract:*',
                       'in': 'state:{} county:085'.format(states.CA.fips)})


In [ ]:
c.acs5.state_zipcode("B01001_004E", Census.ALL, 94040)

In [ ]:
c.acs5.state_zipcode("B01001_004E", states.CA.fips, 94040)

In [ ]:
result = censusgeocode.onelineaddress("2645 California Street, Mountain View, CA")

In [ ]:
result

In [ ]:
states.CA.fips


In [ ]:
c.acs5.state_county_blockgroup("B01003_001E", states.CA.fips, "085", "3", "509401")

In [ ]:
api = USCensusAPI(CENSUS_API_KEY, verbose=True)
api.warm_cache()
api.get_census_data_from_address("B01003_001E", "2685 California Street, Mountain View, CA")
api.get_census_data_from_lat_long("B01003_001E", 37.4069801, -122.1144634)

In [ ]:
censusgeocode.coordinates(37.4069801, -122.1144634)